In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import warnings
import gspread
warnings.filterwarnings('ignore')

In [2]:
gc = gspread.service_account(filename = "C:\Data Science\KNN\Knn_gender_height_weight\linear-regression-404916-8d9c449085dc.json")

In [3]:
sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/10Rn9DxaOxs_1oiQwymWyjf47nc6d2vLCam2l6stDQgY/edit?usp=sharing")

In [4]:
ws = sheet.worksheet('weight-height')

In [5]:
df = pd.DataFrame(ws.get_all_records())

In [6]:
df

,Gender,Height,Weight
0,Male,73.847017,241.893563
1,Male,68.781904,162.310473
2,Male,74.110105,212.740856
3,Male,71.730978,220.042470
4,Male,69.881796,206.349801
...,...,...,...
8550,Female,60.483946,110.565497
8551,Female,63.423372,129.921671
8552,Female,65.584057,155.942671
8553,Female,67.429971,151.678405


In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
ohe = OneHotEncoder(sparse=False, drop='first')

In [9]:
gender_ohe = ohe.fit_transform(df[['Gender']])

In [10]:
gender_ohe


array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

In [11]:
df.drop(columns='Gender', inplace=True)

In [12]:
df.values

array([[ 73.84701702, 241.8935632 ],
       [ 68.78190405, 162.3104725 ],
       [ 74.11010539, 212.7408556 ],
       ...,
       [ 65.58405676, 155.9426708 ],
       [ 67.42997138, 151.6784055 ],
       [ 60.92179087, 131.2537378 ]])

In [13]:
combined_values = np.hstack((df[['Height', 'Weight']].values,gender_ohe,))

In [14]:
combined_values

array([[ 73.84701702, 241.8935632 ,   1.        ],
       [ 68.78190405, 162.3104725 ,   1.        ],
       [ 74.11010539, 212.7408556 ,   1.        ],
       ...,
       [ 65.58405676, 155.9426708 ,   0.        ],
       [ 67.42997138, 151.6784055 ,   0.        ],
       [ 60.92179087, 131.2537378 ,   0.        ]])

In [18]:
encoded_column_names = ohe.get_feature_names_out(['Gender'])
columns =  ['Height', 'Weight']+list(encoded_column_names) 

In [19]:
combined_df = pd.DataFrame(combined_values, columns=columns)

In [20]:
combined_df

,Height,Weight,Gender_Male
0,73.847017,241.893563,1.0
1,68.781904,162.310473,1.0
2,74.110105,212.740856,1.0
3,71.730978,220.042470,1.0
4,69.881796,206.349801,1.0
...,...,...,...
8550,60.483946,110.565497,0.0
8551,63.423372,129.921671,0.0
8552,65.584057,155.942671,0.0
8553,67.429971,151.678405,0.0


In [24]:
from sklearn.model_selection import train_test_split

In [25]:
x = combined_df.drop('Gender_Male', axis=1)
y = combined_df[['Gender_Male']]

In [26]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.3)

# KNN Classifier

In [27]:
 from sklearn.neighbors import KNeighborsClassifier

In [28]:
neigh = KNeighborsClassifier(n_neighbors=3)

In [29]:
 neigh.fit(xtrain, ytrain)

KNeighborsClassifier(n_neighbors=3)

In [31]:
 neigh.predict(xtest)

array([0., 1., 0., ..., 1., 1., 1.])

In [33]:
ytest['predicted_y'] = neigh.predict(xtest)

In [34]:
ytest

,Gender_Male,predicted_y
5950,0.0,0.0
5609,0.0,1.0
7588,0.0,0.0
4981,1.0,1.0
1685,1.0,1.0
...,...,...
7048,0.0,0.0
7440,0.0,0.0
3766,1.0,1.0
1755,1.0,1.0


In [37]:
neigh.score(xtrain, ytrain)

0.9387107548430194

In [39]:
ytest.drop('predicted_y', axis=1, inplace=True)

In [41]:
neigh.score(xtest, ytest)

0.9037787300350604